In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
import pydicom
from glob import glob
from tqdm.notebook import tqdm
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
from skimage import exposure
import cv2
import warnings
warnings.filterwarnings("ignore")
from skimage import exposure

In [ ]:
import matplotlib.patches as ptc
from PIL import Image
import albumentations

In [ ]:
# Helper function for reading a DICOM file. 

def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    # not sure why this is done? 
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def plot_img(img_path, size = (7,7), cmap = 'gray'):
    img = read_xray(img_path)
    plt.figure(figsize=size)
    plt.imshow(img, cmap = cmap)
    plt.show()
    
def view_with_annot(image_name, train_csv, CLAHE = False):
    if CLAHE == True:
        img = read_xray(f"../input/vinbigdata-chest-xray-abnormalities-detection/train/{image_name}.dicom")
        img = exposure.equalize_hist(img)
    else: 
        img = read_xray(f"../input/vinbigdata-chest-xray-abnormalities-detection/train/{image_name}.dicom")
        
    fig, ax = plt.subplots(1, figsize = (10,10))
    ax.imshow(img, cmap = 'gray')

    for i in range(0,len(train_csv[train_csv['image_id']==image_name])):
        rect = ptc.Rectangle((train_csv[train_csv['image_id']==image_name].reset_index(drop = True).x_min[i], 
                              train_csv[train_csv['image_id']==image_name].reset_index(drop = True).y_min[i]), 
                             train_csv[train_csv['image_id']==image_name].reset_index(drop = True).x_max[i]-train_csv[train_csv['image_id']==image_name].reset_index(drop = True).x_min[i], 
                             train_csv[train_csv['image_id']==image_name].reset_index(drop = True).y_max[i]-train_csv[train_csv['image_id']==image_name].reset_index(drop = True).y_min[i], 
                             linewidth = 1, edgecolor = cols[train_csv[train_csv['image_id']==image_name].class_id.reset_index(drop = True)[i]], facecolor = 'none')
        ax.add_patch(rect)
        ax.annotate(train_csv[train_csv['image_id']==image_name].reset_index(drop = True).class_name[i],
                    (train_csv[train_csv['image_id']==image_name].reset_index(drop = True).x_min[i], 
                    train_csv[train_csv['image_id']==image_name].reset_index(drop = True).y_min[i] - 24) )
        
cols = ['#e41a1c', '#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628','#f781bf','#999999', '#000000', '#1b9e77', '#d95f02', '#7570b3', '#e7298a']

In [ ]:
#Paths
train_original_data = "../input/vinbigdata-chest-xray-abnormalities-detection/train"
test_original_data = "../input/vinbigdata-chest-xray-abnormalities-detection/test"
train_csv_orig_path = "../input/vinbigdata-chest-xray-abnormalities-detection/train.csv" 

In [ ]:
print("The number of Dicom images in train: ", len(os.listdir(train_original_data)))
print("The number of Dicom images in test: ", len(os.listdir(test_original_data)))

In [ ]:
train_csv_orig = pd.read_csv(train_csv_orig_path)
train_csv_orig.head()

In [ ]:
#lets plot images with annotations
#one with No findings and other with findings

eg_1_path = '21a10246a5ec7af151081d0cd6d65dc9.dicom'
eg_2_path = '051132a778e61a86eb147c7c6f564dfe.dicom'
eg_1 = read_xray(os.path.join(train_original_data, eg_1_path))
eg_2 = read_xray(os.path.join(train_original_data, eg_2_path))


f = plt.figure(figsize=(10,10))
ax1 = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax1.imshow(eg_1,cmap = 'gray')
ax1.set_title("Example_1")
ax2.imshow(eg_2, cmap = 'gray')
ax2.set_title("Example_2")


f2 = plt.figure(figsize=(10,10))
ax1 = f2.add_subplot(121)
ax2 = f2.add_subplot(122)
ax1.imshow(exposure.equalize_hist(eg_1),cmap = 'gray')
ax1.set_title("Example_1_CLAHE")
ax2.imshow(exposure.equalize_hist(eg_2), cmap = 'gray')
ax2.set_title("Example_2_CLAHE")

In [ ]:
# Ploting Annotatations
view_with_annot("9a5094b2563a1ef3ff50dc5c7ff71345", train_csv=train_csv_orig, CLAHE = False)
view_with_annot("9a5094b2563a1ef3ff50dc5c7ff71345", train_csv=train_csv_orig, CLAHE = True)

### Finding resizing Approach

In [ ]:
#Creating a resized image dataset 512X512
from PIL import Image 
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im


In [ ]:
def resize_image(img_arr, bboxes, h, w):
    """
    :param img_arr: original image as a numpy array
    :param bboxes: bboxes as numpy array where each row is 'x_min', 'y_min', 'x_max', 'y_max', "class_id"
    :param h: resized height dimension of image
    :param w: resized weight dimension of image
    :return: dictionary containing {image:transformed, bboxes:['x_min', 'y_min', 'x_max', 'y_max', "class_id"]}
    """
    # create resize transform pipeline
    transform = albumentations.Compose(
        [albumentations.Resize(height=h, width=w, always_apply=True)],
        bbox_params=albumentations.BboxParams(format='pascal_voc'))

    transformed = transform(image=img_arr, bboxes=bboxes)

    return transformed


# transformed_dict = resize_image(sample_arr, bboxes_og, 224, 224)

# # contains the image as array
# transformed_arr = transformed_dict["image"]

# # contains the resized bounding boxes
# transformed_info = np.array(list(map(list, transformed_dict["bboxes"]))).astype(float)

In [ ]:
bboxes_og = train_csv_orig[train_csv_orig.image_id == '051132a778e61a86eb147c7c6f564dfe'][['x_min', 'y_min', 'x_max', 'y_max', 'class_id']].values

In [ ]:
transformed_dict = resize_image(eg_2, bboxes_og, 512, 512)

In [ ]:
transformed_info = np.array(list(map(list, transformed_dict["bboxes"]))).astype(float)

In [ ]:
transformed_info

In [ ]:
fig, ax = plt.subplots(1, figsize = (10,10))
ax.imshow(transformed_dict['image'], cmap = 'gray')
for i in transformed_info:
    rect = ptc.Rectangle((i[0], i[1]), i[2]-i[0], i[3]-i[1], 
                        linewidth = 1, edgecolor = cols[int(i[4])], 
                         facecolor = 'none')
    ax.add_patch(rect)
    
    
    

In [ ]:
view_with_annot('051132a778e61a86eb147c7c6f564dfe', train_csv=train_csv_orig)

In [ ]:
train_csv_orig[train_csv_orig['image_id']=='051132a778e61a86eb147c7c6f564dfe'].class_id.reset_index(drop = True)[int(i[4])

In [ ]:
print(eg_2.shape)
plt.imshow(eg_2, cmap = 'gray')

In [ ]:
eg_2_resized = resize(eg_1, size = 512)
plt.imshow(eg_2_resized, cmap = 'gray')

In [ ]:
train_csv_orig[train_csv_orig.image_id == '051132a778e61a86eb147c7c6f564dfe'][['x_min', 'y_min', 'x_max', 'y_max']].values//(2880/512, 2304/512,2880/512, 2304/512)

In [ ]:
fig, ax = plt.subplots(1)
ax.imshow(eg_2_resized, cmap = 'gray')
image_name = '051132a778e61a86eb147c7c6f564dfe'
for i in range(0,len(train_csv_orig[train_csv_orig['image_id']==image_name])):
    rect = ptc.Rectangle((train_csv_orig[train_csv_orig['image_id']==image_name].reset_index(drop = True).x_min[i]//(2880/512), 
                          train_csv_orig[train_csv_orig['image_id']==image_name].reset_index(drop = True).y_min[i]//(2304/512)), 
                         train_csv_orig[train_csv_orig['image_id']==image_name].reset_index(drop = True).x_max[i]//(2880/512)-train_csv_orig[train_csv_orig['image_id']==image_name].reset_index(drop = True).x_min[i]//(2880/512), 
                         train_csv_orig[train_csv_orig['image_id']==image_name].reset_index(drop = True).y_max[i]//(2304/512)-train_csv_orig[train_csv_orig['image_id']==image_name].reset_index(drop = True).y_min[i]//(2304/512), 
                         linewidth = 1, edgecolor = cols[train_csv_orig[train_csv_orig['image_id']==image_name].class_id.reset_index(drop = True)[i]], facecolor = 'none')
    ax.add_patch(rect)
    ax.annotate(train_csv_orig[train_csv_orig['image_id']==image_name].reset_index(drop = True).class_name[i],
                (train_csv_orig[train_csv_orig['image_id']==image_name].reset_index(drop = True).x_min[i]//(2880/512), 
                train_csv_orig[train_csv_orig['image_id']==image_name].reset_index(drop = True).y_min[i]//(2304/512) - 24) )

In [ ]:
transformed_dict['image']

In [ ]:
np.array(eg_2_resized)